# PROJECT ONE: Data Analysis
### Cecilia Cabiya

In [2]:
from dotenv import load_dotenv
import os
import requests
import json
import datetime
import time
import pandas as pd

load_dotenv()

polygon_key = os.environ['polygon_key']
server_name = os.environ['server_name']

base_auth_url= 'https://api.polygon.io/'
# Sample URL for aggregations over a given time period 
agg_url_sample = r'/v2/aggs/ticker/{stocksTicker}/range/{multiplier}/{timespan}/{from}/{to}'
# Sample URL for Grouped daily 
day_url_sample = r'/v2/aggs/grouped/locale/us/market/stocks/{date}'

### RETRIEVING STOCK DATA

In [43]:
stocks = {
    'AMZN':'Amazon.com Inc',
    'TSLA':'Tesla Inc',
    'BIP':'Brookfield Infrastructure Partners L.P.',
    'NFLX':'Netflix Inc',
    'VTI': 'Vanguard Total Stock Market ETF',
    'VYM': 'Vanguard High Dividend Yield ETF' # One of the ETFs
}


In [44]:
# Run every time for every stock
stock = 'VYM' # The stock you want
start_date = datetime.date(2023,1,1)
end_date = datetime.date(2024,7,1)

In [ ]:
# Run every time for every stock
agg_url_ext = f'v2/aggs/ticker/{stock}/range/1/day/{start_date}/{end_date}'

params = {"apiKey": polygon_key}

full_url = base_auth_url+agg_url_ext
response = (requests.get(full_url, params = params)).json()

try:
    results = response['results']
except:
    print(response)

for i, res in enumerate(results):
    timems = results[i]['t']
    dt = datetime.date.fromtimestamp((timems/1000.0))
    res['date'] = dt.strftime('%Y-%m-%d')
    res['stock'] = stock
    res['stock_company'] = stocks[stock]

df = pd.DataFrame.from_records(results)
df.head()

In [13]:
# Run every time for every stock
# Renaming the dataframe's columns
df.rename(columns={'v': 'volume', 'vw': 'volume_weighted',
                   't': 'timestamp', 'o': 'open_price',
                   'n': 'transactions', 'c': 'close_price'
                   , 'h': 'highest_price'
                   , 'l': 'lowest_price'}, inplace=True)

# Dropping timestamp, as we already have date
df.drop(columns = ['timestamp'])

# Moving around the columns
df = df[['stock', 'stock_company','date','open_price','close_price','highest_price',
    'lowest_price','transactions','volume','volume_weighted']]

In [3]:
# Run every time for safety
import pymssql

conn = pymssql.connect(server=server_name)

# Create a cursor to run queries
cursor = conn.cursor()

In [4]:
# Run every time for safety
cursor.execute("USE stock_project")

In [ ]:
# Run only once RAN
cursor.execute("""CREATE TABLE stocks_table(
               stock_id int PRIMARY KEY IDENTITY,
               stock VARCHAR(10),
               company_name VARCHAR(200)
               );""")

cursor.execute("""CREATE TABLE dates_table(
               date_id int PRIMARY KEY IDENTITY,
               dates_date DATETIME,
               dates_year INT,
               dates_month INT,
               dates_day INT,
               );""")

In [12]:
# Only run once RAN
# Creating the fact table 
cursor.execute("""CREATE TABLE fact_table(
               fact_id int PRIMARY KEY IDENTITY,
               fact_stock_id INT FOREIGN KEY REFERENCES stocks_table(stock_id),
               fact_date_id INT FOREIGN KEY REFERENCES dates_table(date_id),
               open_price INT,
               close_price INT,
               highest_price INT, 
               lowest_price INT,
               volume INT,
               vwap INT,
               transactions INT
               );""")

In [126]:
# Only adding the dates once, as all stocks will not have unique date data; they're all 
#   reported on the same days RAN
dates = pd.to_datetime(df['date'])
years = dates.dt.year
months = dates.dt.month
days = dates.dt.day


date_frame = {"Date":dates,
              "Year": years,
              "Month": months,
              "Day": days}
date_frame = pd.DataFrame(date_frame)
date_frame.head()

query = """INSERT INTO dates_table
           VALUES (%s,%d,%d,%d)"""

cursor.executemany(query, date_frame.values.tolist())

In [16]:
# This will be run every time for every stock 
query = f'''INSERT INTO stocks_table
           VALUES ('{stock}','{stocks[stock]}')'''
cursor.execute(query)

In [17]:
# Run every time for every stock
df = df[['stock', 'date','open_price','close_price','highest_price',
    'lowest_price','volume','volume_weighted','transactions']]
query = """INSERT INTO fact_table (fact_stock_id, fact_date_id,
                                     open_price, close_price, highest_price, lowest_price,
                                     volume, vwap, transactions)
           VALUES (
                (SELECT stock_id FROM stocks_table
                WHERE stock = %s),
                (SELECT date_id FROM dates_table
                WHERE FORMAT(dates_date, 'yyyy-MM-dd') = %s),
                %d,
                %d,
                %d,
                %d,
                %d,
                %d,
                %d
                );"""

cursor.executemany(query, df.values.tolist())

In [ ]:
# Run every time for every stock
conn.commit()

### RETRIEVING FOREX DATA

In [176]:
# On forex
# A minor note; due to the fact that Europe and US have different timezones, beginning at 
#   a start date will make the day before the start date as a beginning
currencies = 'C:EURUSD' # The foreign conversion
start_date = datetime.date(2023,1,2)
end_date = datetime.date(2024,7,1)

base_auth_url= 'https://api.polygon.io/'

In [177]:
# Run every time for every stock
forex_url = f'v2/aggs/ticker/{currencies}/range/1/day/{start_date}/{end_date}'

params = {"apiKey": polygon_key}

full_forex_url = base_auth_url+forex_url
forex_response = (requests.get(full_forex_url, params = params)).json()

try:
    forex_results = forex_response['results']
except:
    print(forex_response)

for i, res in enumerate(forex_results):
    timems = forex_results[i]['t']
    dt = datetime.date.fromtimestamp((timems/1000.0))
    res['date'] = dt.strftime('%Y-%m-%d')

df = pd.DataFrame.from_records(forex_results)

# Renaming the dataframe's columns
df.rename(columns={'v': 'volume', 'vw': 'volume_weighted',
                   'o': 'open_price',
                   'n': 'transactions', 'c': 'close_price'
                   , 'h': 'highest_price'
                   , 'l': 'lowest_price'}, inplace=True)

# Dropping timestamp (t), as we already have date
df.drop(columns = ['t'], inplace=True)

# Moving around the columns
df = df[['date', 'volume','volume_weighted','transactions','open_price','close_price','highest_price','lowest_price']]

df.head()

,date,volume,volume_weighted,transactions,open_price,close_price,highest_price,lowest_price
0,2023-01-01,74147,1.0676,74147,1.07027,1.06764,1.07100,1.06450
1,2023-01-02,236637,1.0585,236637,1.06765,1.05450,1.06810,1.05180
2,2023-01-03,229494,1.0598,229494,1.05453,1.06050,1.06353,1.05440
3,2023-01-04,210543,1.0574,210543,1.06056,1.05211,1.06320,1.05138
4,2023-01-05,207800,1.0551,207800,1.05209,1.06610,1.06620,1.04829


In [171]:
# RUN ONCE RAN

cursor.execute('USE stock_project')

cursor.execute('''CREATE TABLE forex_table(
               forex_date DATETIME,
               forex_volume INT,
               forex_vwap DECIMAL(18,5),
               forex_transactions INT,
               forex_open_price DECIMAL(18,5),
               forex_close_price DECIMAL(18,5),
               forex_highest_price DECIMAL(18,5), 
               forex_lowest_price DECIMAL(18,5),
               forex_diff DECIMAL(18,5),
               forex_percent_inc DECIMAL(18,3));''')

In [179]:
# RUN ONCE RAN
query = '''INSERT INTO forex_table (forex_date, forex_volume, forex_vwap, forex_transactions,
              forex_open_price, forex_close_price, forex_highest_price, forex_lowest_price)
           VALUES(%s, %d, %d, %d,
                  %d, %d, %d, %d)'''

cursor.executemany(query, df.values.tolist())

In [180]:
# RUN ONCE
cursor.execute("""UPDATE forex_table 
               SET forex_diff = forex_open_price - (SELECT TOP 1 forex_open_price 
                                                    FROM forex_table)""")
cursor.execute("""UPDATE forex_table 
               SET forex_percent_inc = CAST(forex_diff AS DECIMAL(18,2))*100/(
                                SELECT TOP 1 forex_open_price 
                                FROM forex_table)""")


In [ ]:
conn.commit()

### MAKING STOCK PRICE INFORMATION SUBTABLES

In [27]:
# Make a subtable for every stock and etf
cursor.execute('''CREATE TABLE vym_prices_over_time(
                vti_id INT PRIMARY KEY IDENTITY, 
                dates_date DATETIME,
                open_price INT,
                close_price INT,
                highest_price INT, 
                lowest_price INT, 
                diff INT,
                percen_inc DECIMAL(18,2));''')

In [31]:
cursor.execute("""INSERT INTO vym_prices_over_time (dates_date, open_price, 
               close_price, highest_price, lowest_price)
               SELECT 
               dates_date, open_price, close_price, highest_price, lowest_price
               FROM fact_table JOIN dates_table ON fact_date_id = date_id
               WHERE fact_stock_id = 8""")

In [32]:
cursor.execute("""UPDATE vym_prices_over_time 
               SET diff = open_price - (SELECT TOP 1 open_price FROM vym_prices_over_time)
               """)
cursor.execute("""UPDATE vym_prices_over_time 
               SET percen_inc =
                (CAST(diff AS  DECIMAL(18,2))*100/(SELECT TOP 1 open_price FROM vym_prices_over_time))
               """)

In [39]:
# RUN ONCE RAN
cursor.execute('''CREATE TABLE stock_volatility_table(
                volatility_id int PRIMARY KEY IDENTITY,
                vol_stock_id INT FOREIGN KEY REFERENCES stocks_table(stock_id),
                risk_free_rate DECIMAL(10,2),
                inflation DECIMAL(10,2),
                price_variance DECIMAL(18,2), 
                return_variance DECIMAL(18,2),
                price_stdev DECIMAl(18,2),
                return_stdev DECIMAL(18,2),
                beta DECIMAL(18,2), 
                sharpe DECIMAL(18,2));''') 

In [41]:
# RUN ONCE RAN
cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES( 1,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM amazon_price_over_time),
                (SELECT VAR(diff) FROM amazon_price_over_time),
                (SELECT STDEV(open_price) FROM amazon_price_over_time),
                (SELECT STDEV(diff) FROM amazon_price_over_time),
                1.15
               )''')

In [43]:
# RUN ONCE RAN
cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM amazon_price_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 1)
               WHERE vol_stock_id = 1''')

In [45]:
# RUN ONCE RAN
cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES( 5,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM bip_price_over_time),
                (SELECT VAR(diff) FROM bip_price_over_time),
                (SELECT STDEV(open_price) FROM bip_price_over_time),
                (SELECT STDEV(diff) FROM bip_price_over_time),
                1.03
               )''')

cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM bip_price_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 5)
               WHERE vol_stock_id = 5''')

cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES(6,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM tesla_price_over_time),
                (SELECT VAR(diff) FROM tesla_price_over_time),
                (SELECT STDEV(open_price) FROM tesla_price_over_time),
                (SELECT STDEV(diff) FROM tesla_price_over_time),
                2.33
               )''')

cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM tesla_price_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 6)
               WHERE vol_stock_id = 6''')

cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES(4,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM netflix_price_over_time),
                (SELECT VAR(diff) FROM netflix_price_over_time),
                (SELECT STDEV(open_price) FROM netflix_price_over_time),
                (SELECT STDEV(diff) FROM netflix_price_over_time),
                1.27
               )''')

cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM netflix_price_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 4)
               WHERE vol_stock_id = 4''')

In [48]:
cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES(7,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM vti_prices_over_time),
                (SELECT VAR(diff) FROM vti_prices_over_time),
                (SELECT STDEV(open_price) FROM vti_prices_over_time),
                (SELECT STDEV(diff) FROM vti_prices_over_time),
                1.01
               )''')

cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM vti_prices_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 7)
               WHERE vol_stock_id = 7''')

cursor.execute('''INSERT INTO stock_volatility_table (vol_stock_id,
               risk_free_rate, inflation, price_variance, return_variance,
               price_stdev, return_stdev, beta)
               VALUES(8,
                4.28,
                3.2, 
                (SELECT VAR(open_price) FROM vym_prices_over_time),
                (SELECT VAR(diff) FROM vym_prices_over_time),
                (SELECT STDEV(open_price) FROM vym_prices_over_time),
                (SELECT STDEV(diff) FROM vym_prices_over_time),
                .76
               )''')

cursor.execute(''' UPDATE stock_volatility_table
               SET sharpe = ((SELECT TOP 1 diff 
                            FROM vym_prices_over_time 
                            ORDER BY dates_date DESC) - risk_free_rate)/(SELECT return_stdev
                                                                FROM stock_volatility_table
                                                                WHERE vol_stock_id = 8)
               WHERE vol_stock_id = 8''')

In [ ]:
cursor.execute('SELECT  * from stock_volatility_table')
rows = cursor.fetchall()
[print(row) for row in rows]

### POPULATING THE INDECES DATA

In [105]:
indices = {
    'NDXT':'NASDAQ-100 Technology Sector',
    'NDX':'Nasdaq-100'
}
index_marker = 'NDXT'

In [ ]:
# For indices, not stocks

index_url_ext = f'v2/aggs/ticker/I:{index_marker}/range/1/day/{start_date}/{end_date}'
params = {"apiKey": polygon_key}
full_ind_url = base_auth_url + index_url_ext
index_response = (requests.get(full_ind_url, params = params)).json()

index_response

In [ ]:
# Run once for each index
try:
    ind_results = index_response['results']
except:
    print(index_response)

for i, res in enumerate(ind_results):
    timems = ind_results[i]['t']
    dt = datetime.date.fromtimestamp((timems/1000.0))
    res['date'] = dt.strftime('%Y-%m-%d')
    res['index_marker'] = index_marker
ind_df = pd.DataFrame.from_records(ind_results)
ind_df.drop(columns = ['t'], inplace=True)
ind_df = ind_df[['index_marker','date','o', 'c', 'h', 'l']]
ind_df.rename(columns={'date':'date',
                       'o':'open_price',
                       'c':'close_price',
                       'h':'highest_price',
                       'l':'lowest_price'}, inplace=True)
ind_df

In [ ]:
# RUN ONCE .
cursor.execute('''CREATE TABLE index_table(
                index_id int PRIMARY KEY IDENTITY,
                index_marker VARCHAR(6),
                company VARCHAR(100))''')

In [61]:
# RUN ONCE .
cursor.execute('''INSERT INTO index_table 
                VALUES ('NDX','NASDAQ-100')''')
cursor.execute('''INSERT INTO index_table 
                VALUES ('NDXT','NASDAQ-100 Technology Sector')''')

In [ ]:
# RUN ONCE .
cursor.execute('''CREATE TABLE market_index_growth(
                market_id int PRIMARY KEY IDENTITY,
                market_index_id INT FOREIGN KEY REFERENCES index_table(index_id),
                market_index_date DATETIME,
                market_open_value DECIMAL(18,5),
                market_close_value DECIMAL(18,5),
                market_highest_value DECIMAL(18,5),
                market_lowest_value DECIMAL(18,5))''')
cursor.execute('''CREATE TABLE price_index_growth(
                price_id int PRIMARY KEY IDENTITY,
                price_index_id INT FOREIGN KEY REFERENCES index_table(index_id),
                price_index_date DATETIME,
                price_open_value DECIMAL(18,5),
                price_close_value DECIMAL(18,5),
                price_highest_value DECIMAL(18,5),
                price_lowest_value DECIMAL(18,5))''')

In [101]:
# RUN ONCE .
query = """INSERT INTO market_index_growth
        VALUES (
            (SELECT index_id FROM index_table 
            WHERE index_marker = %s),
            %s,
            %s,
            %s,
            %s,
            %s)"""
cursor.executemany(query, ind_df.values.tolist())

In [102]:
# RUN ONCE .
cursor.execute("""ALTER TABLE market_index_growth 
                ADD diff DECIMAL(18,5), percent_inc DECIMAL(18,2)""")

In [ ]:
# RUN ONCE .
cursor.execute('''ALTER TABLE market_index_growth ADD diff_from_avg_percent DECIMAL(18,5)''')
cursor.execute('''ALTER TABLE price_index_growth ADD diff_from_avg_percent DECIMAL(18,5)''')

In [103]:
# RUN ONCE .
cursor.execute("""UPDATE market_index_growth
               SET diff = market_open_value - (SELECT TOP 1 market_open_value FROM market_index_growth)""")

In [104]:
# RUN ONCE .
cursor.execute("""UPDATE market_index_growth
               SET percent_inc = 
               CAST(diff AS DECIMAL(18,2))*100/(SELECT TOP 1 market_open_value FROM market_index_growth)""")

In [108]:
# RUN ONCE .
query = """INSERT INTO price_index_growth
        VALUES (
            (SELECT index_id FROM index_table 
            WHERE index_marker = %s),
            %s,
            %s,
            %s,
            %s,
            %s)"""
cursor.executemany(query, ind_df.values.tolist())

In [109]:
# RUN ONCE .
cursor.execute("""ALTER TABLE price_index_growth 
                ADD diff DECIMAL(18,5), percent_inc DECIMAL(18,2)""")
cursor.execute("""UPDATE price_index_growth
               SET diff = price_open_value - (SELECT TOP 1 price_open_value FROM price_index_growth)""")
cursor.execute("""UPDATE price_index_growth
               SET percent_inc = 
               CAST(diff AS DECIMAL(18,2))*100/(SELECT TOP 1 price_open_value FROM price_index_growth)""")

In [27]:
# RUN ONCE .
cursor.execute("""UPDATE market_index_growth
               SET diff_from_avg_percent = percent_inc-(SELECT AVG(percent_inc) FROM market_index_growth)""")

cursor.execute("""UPDATE price_index_growth
               SET diff_from_avg_percent = percent_inc-(SELECT AVG(percent_inc) FROM price_index_growth)""")

In [ ]:
conn.commit()

### POPULATING INVESTMENT DATA

In [13]:
# RUN ONCE .
cursor.execute(''' CREATE TABLE investment_table(
               investment_id INT PRIMARY KEY IDENTITY,
               inv_amt INT,
               inv_date DATETIME,
               am_amount INT,
               am_return DECIMAL (18,2),
               bip_amount INT,
               bip_return DECIMAL (18,2),
               tesla_amount INT,
               tesla_return DECIMAL (18,2),
               netflix_amount INT,
               netflix_return DECIMAL (18,2),
               vti_amount INT,
               vti_return DECIMAL(18,2),
               vym_amount INT,
               vym_return DECIMAL(18,2),
               forex_amount INT,
               forex_return DECIMAL(18,2),
               total_return DECIMAL (18,2)
               )''')

In [5]:
# First investment entirely in stocks, beginning 2023 01 01
#   am = 30 bip = 30 tesla = 10 netflix = 30 rest 0 bc they want to focus on stocks
# Second investment trying to equal out the % given totally
#   all at 15 except forex, which is at 10
# Third investment marks a more informed stock decision based on number one
#   Amazon, tesla, netflix, vti, and vym all at 20

investment = 100000 
percent_amazon = .30
percent_bip = 0
percent_tesla = 0
percent_netflix = .30
percent_vti = .30
percent_vym = .10
percent_forex = 0
investment_date = datetime.date(2023,1,1)


In [6]:
amazon_inv = investment*percent_amazon
bip_inv = investment*percent_bip
tesla_inv = investment*percent_tesla
netflix_inv = investment*percent_netflix
vti_inv = investment*percent_vti
vym_inv = investment*percent_vym
forex_inv = investment*percent_forex

query_params = [investment, investment_date, amazon_inv, bip_inv, tesla_inv, netflix_inv, vti_inv, vym_inv, forex_inv]
query = """INSERT INTO investment_table(inv_amt, inv_date, am_amount, bip_amount, tesla_amount, netflix_amount,
            vti_amount, vym_amount, forex_amount)
            VALUES (%s, %s, %d, %d, %d, %d, %d, %d, %d)"""
cursor.execute(query, query_params)

cursor.execute("""UPDATE investment_table
                SET am_return = am_amount * (SELECT TOP 1 percen_inc/100
                                          FROM amazon_price_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET bip_return = bip_amount * (SELECT TOP 1 percen_inc/100
                                          FROM bip_price_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET tesla_return = tesla_amount * (SELECT TOP 1 percen_inc/100
                                          FROM tesla_price_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET netflix_return = netflix_amount * (SELECT TOP 1 percen_inc/100
                                          FROM netflix_price_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET vti_return = vti_amount * (SELECT TOP 1 percen_inc/100
                                          FROM vti_prices_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET vym_return = vym_amount * (SELECT TOP 1 percen_inc/100
                                          FROM vym_prices_over_time a
                                          ORDER BY a.dates_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET forex_return = forex_amount * (SELECT TOP 1 forex_percent_inc/100
                                          FROM forex_table a
                                          ORDER BY a.forex_date DESC)""")
cursor.execute("""UPDATE investment_table
                SET total_return = am_return+bip_return+tesla_return+netflix_return+
                vti_return+vym_return+forex_return""")

In [ ]:
cursor.execute(''' SELECT * from market_index_growth''')
rows = cursor.fetchall()
[print(row) for row in rows]

In [7]:
conn.commit()

### MOVING AVERAGES

In [ ]:
# Sorry about the long query, but this displays all the hundred
#   and ten day averages for all stocks and etfs, alongside indices
cursor.execute("""CREATE TABLE moving_averages(
	moving_avg_id INT PRIMARY KEY IDENTITY,
	dates_date DATETIME, 
	ten_day_amazon DECIMAL(18,2),
	hundred_day_amazon DECIMAL(18,2),
	ten_day_bip DECIMAL(18,2),
	hundred_day_bip DECIMAL(18,2),
	ten_day_tesla DECIMAL(18,2),
	hundred_day_tesla DECIMAL(18,2),
	ten_day_netflix DECIMAL(18,2),
	hundred_day_netflix DECIMAL(18,2),
	ten_day_vti DECIMAL(18,2),
	hundred_day_vti DECIMAL(18,2),
	ten_day_vym DECIMAL(18,2),
	hundred_day_vym DECIMAL(18,2),
	ten_day_market DECIMAL(18,2),
	hundred_day_market DECIMAL(18,2),
	ten_day_price DECIMAL(18,2),
	hundred_day_price DECIMAL(18,2)
)""")

cursor.execute("""INSERT INTO moving_averages
SELECT a.dates_date, 
AVG(a.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(a.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(b.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(b.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(t.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(t.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(n.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(n.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(vti.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING), 
AVG(vti.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(vyu.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(vyu.open_price) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(market_open_value) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(market_open_value) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING),
AVG(price_open_value) OVER (ORDER BY a.dates_date ROWS BETWEEN 5 PRECEDING AND 4 FOLLOWING),
AVG(price_open_value) OVER (ORDER BY a.dates_date ROWS BETWEEN 50 PRECEDING AND 49 FOLLOWING)
FROM 
(amazon_price_over_time a JOIN bip_price_over_time b 
ON a.dates_date = b.dates_date
JOIN tesla_price_over_time t ON a.dates_date = t.dates_date
JOIN netflix_price_over_time n ON t.dates_date = n.dates_date
JOIN vti_prices_over_time vti on vti.dates_date = n.dates_date
JOIN vym_prices_over_time vyu ON vyu.dates_date = vti.dates_date
JOIN market_index_growth m ON t.dates_date = m.market_index_date
JOIN price_index_growth p ON p.price_index_date = m.market_index_date)
""")